In [1]:
import pandas as pd
from transformers import pipeline

2025-04-13 17:15:38.373273: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-13 17:15:38.384209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744557338.397607  252382 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744557338.401366  252382 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744557338.411386  252382 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
df = pd.read_csv('../data/df_unique.csv')
df

,beer_id,username,date,text,look,smell,taste,feel,overall,score,sentiment,vader_prediction
0,271781,bluejacket74,2017-03-17,"750 ml bottle, 2016 vintage, bottle #304 of 36...",4.00,4.00,4.0,4.25,4.00,4.03,positive,positive
1,125646,GratefulBeerGuy,2017-12-20,0% 16 oz can. Funny story: As I finally walked...,4.75,4.75,4.5,4.50,4.50,4.58,positive,positive
2,150672,KingHoppy,2015-08-14,"Passing through Swift Current SK, I stopped at...",4.00,4.00,4.5,4.25,4.25,4.28,positive,positive
3,150672,biboergosum,2014-12-26,"355ml can, the latest from this nascent craft ...",3.75,4.25,4.0,3.75,3.75,3.97,neutral,positive
4,104824,Fcolle2,2014-05-27,Batch 209 Pours thick dark viscous black in co...,4.25,4.75,4.5,4.75,4.50,4.57,positive,positive
...,...,...,...,...,...,...,...,...,...,...,...,...
1149504,58482,DIM,2009-09-04,a: This was a hazy light copery color with gol...,4.00,4.00,3.5,3.00,3.50,3.60,neutral,positive
1149505,58482,jdaelhousen,2009-08-28,Pour produces a solid contribution of body und...,4.00,5.00,4.5,4.00,4.00,4.44,positive,positive
1149506,58482,beerthulhu,2009-08-24,A: Poured a pumpkin pie orange with gold hight...,4.00,3.00,4.0,3.50,3.50,3.61,neutral,positive
1149507,58482,firkinhophead,2009-08-21,"pours a hazy, almost cloudy, autumn gold. Appa...",3.50,4.00,4.0,3.50,4.00,3.92,neutral,positive


In [ ]:
from tqdm import tqdm
tqdm.pandas()

from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def vader_sentiment(text):
    score = sia.polarity_scores(text)['compound']
    match score:
        case _ if score > 0:
            return 'positive'
        case _ if score < 0:
            return 'negative'
        case _ if score == 0:
            return 'neutral'

df['vader_prediction'] = df['text'].swifter.apply(vader_sentiment)

In [ ]:
mismatch_df = df[df['vader_prediction'] != df['sentiment']]
match_df = df[df['vader_prediction'] == df['sentiment']]
mismatch_df.to_csv("../data/mismatch_df.csv", index=False)
match_df.to_csv("../data/match_df.csv", index=False)

In [9]:
classifier = pipeline("sentiment-analysis", truncation=True)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [ ]:
import swifter
from tqdm import tqdm
tqdm.pandas()

df["bert_prediction"] = df["text"].swifter.apply(lambda x: classifier(x)[0]['label'])